# Extend API Testing
## This notebook is for testing the ExtendAPI client manually.

In [1]:
import asyncio
import os
from datetime import datetime, timedelta
from extend import ExtendClient

In [2]:
# Initialize the client
api_key = os.getenv("EXTEND_API_KEY")
api_secret = os.getenv("EXTEND_API_SECRET")
client = ExtendClient(api_key, api_secret)

In [ ]:
print(api_key)
print(api_secret)


In [ ]:
async def test_virtual_cards():
    cards = await client.virtual_cards.get_virtual_cards()
    print("Virtual Cards:")
    print(cards)
    return cards

await test_virtual_cards()

In [ ]:
### Test Transactions API

async def test_transactions():
    txns = await client.transactions.get_transactions()
    print("Transactions:")
    print(txns)
    return txns

await test_transactions()

In [ ]:
async def test_virtual_card_lifecycle():
    """Test the full lifecycle of a virtual card"""
    # Get the first available credit card
    credit_cards = await client.credit_cards.get_credit_cards()
    if not credit_cards.get("creditCards"):
        print("No credit cards available for testing")
        return
    
    credit_card = credit_cards["creditCards"][0]
    print(f"Using credit card: {credit_card['id']}")
    
    # Get test recipient and cardholder from environment
    recipient = os.getenv("EXTEND_TEST_RECIPIENT")
    cardholder = os.getenv("EXTEND_TEST_CARDHOLDER")
    
    # Calculate valid_to date (3 months from today)
    valid_to = (datetime.now() + timedelta(days=90)).strftime("%Y-%m-%dT23:59:59.999Z")
    
    # Create a virtual card
    print("\nCreating virtual card...")
    response = await client.virtual_cards.create_virtual_card(
        credit_card_id=credit_card["id"],
        display_name="Notebook Test Card",
        balance_cents=5000,
        notes="Created by notebook test",
        recurs=False,
        recipient=recipient,
        cardholder=cardholder,
        valid_to=valid_to
    )
    
    card = response["virtualCard"]
    print(f"Created card: {card['id']}")
    print(f"Status: {card['status']}")
    print(f"Balance: ${card['balanceCents']/100:.2f}")
    
    # Get the card details
    print("\nRetrieving card details...")
    get_response = await client.virtual_cards.get_virtual_card_detail(card["id"])
    print(f"Retrieved card: {get_response['virtualCard']['id']}")
    
    # Update the card
    print("\nUpdating card...")
    update_response = await client.virtual_cards.update_virtual_card(
        card_id=card["id"],
        display_name="Updated Notebook Card",
        notes="Updated by notebook test",
        balance_cents=6000
    )
    print(f"Updated card: {update_response['virtualCard']['id']}")
    print(f"New balance: ${update_response['virtualCard']['balanceCents']/100:.2f}")
    
    # Cancel the card
    print("\nCanceling card...")
    cancel_response = await client.virtual_cards.cancel_virtual_card(card["id"])
    print(f"Card status after cancel: {cancel_response['virtualCard']['status']}")
    
    # Close the card (cleanup)
    print("\nClosing card...")
    close_response = await client.virtual_cards.close_virtual_card(card["id"])
    print(f"Final card status: {close_response['virtualCard']['status']}")

await test_virtual_card_lifecycle()